# Find issues in the image dataset

- https://github.com/woctezuma/steam-DINOv2#filtering-optional

## Install packages

In [ ]:
%pip install --quiet cleanvision

## Download the image dataset

In [ ]:
%cd /content

!curl -OL https://github.com/woctezuma/steam-DINOv2/releases/download/input/images_partA.tar.gz
!curl -OL https://github.com/woctezuma/steam-DINOv2/releases/download/input/images_partB.tar.gz

!tar xzf images_partA.tar.gz
!tar xzf images_partB.tar.gz

## Apply `cleanvision`

- https://github.com/cleanlab/cleanvision

In [ ]:
from cleanvision import Imagelab

DATASET_PATH = "images/"
SAVE_PATH = "results"

### Find issues

In [ ]:
find_issues_from_scratch = False

if find_issues_from_scratch:
  imagelab = Imagelab(data_path=DATASET_PATH)
  imagelab.find_issues()
  imagelab.save(SAVE_PATH)

else:
  %cd /content
  !curl -OL https://github.com/woctezuma/steam-DINOv2/releases/download/cleanvision/results.tar.gz
  !tar xzf results.tar.gz

### Report

In [ ]:
imagelab = Imagelab.load(SAVE_PATH, DATASET_PATH)
imagelab.report()

## Analyze results

### Define utility functions

In [ ]:
import json

from pathlib import Path

def save_to_json(data, fname):
  with Path(fname).open('w') as f:
    json.dump(data, f, indent=True)

In [ ]:
IMAGE_SUFFIX = ".jpg"

def to_image_index(image_name,
                   image_suffix=IMAGE_SUFFIX):
    index_as_str = Path(image_name).name.removesuffix(image_suffix)
    return int(index_as_str)

def convert_list_to_image_indices(l, image_suffix=IMAGE_SUFFIX):
  return sorted([to_image_index(fname, image_suffix) for fname in l])

def convert_nested_lists_to_image_indices(nested_lists,
                                          image_suffix=IMAGE_SUFFIX):
  return [ convert_list_to_image_indices(l, image_suffix) for l in nested_lists ]

### Summary

> Dataframe with **global summary** of all issue types detected in your dataset and the overall prevalence of each type.
>
> Reference:
> - https://cleanvision.readthedocs.io/en/latest/tutorials/tutorial.html#imagelab.issue_summary

In [ ]:
imagelab.issue_summary

### Table

> DataFrame **assessing each image** in your dataset, reporting which issues each image exhibits and a score (between 0 and 1):
> - a boolean column per issue type shows whether each image exhibits this issue type.
> - a numeric column assesses the issue severity in each image, where lower values indicate more severe cases.
>
> Reference:
> - https://cleanvision.readthedocs.io/en/latest/tutorials/tutorial.html#imagelab.issues

#### List images with at least one issue

In [ ]:
BOOL_PREFIX = "is_"
SUFFIX_TO_SKIP = "_duplicates_issue"

# Reference: https://stackoverflow.com/a/19483025/376454
bool_headers = [
    header for header in list(imagelab.issues)
    if header.startswith(BOOL_PREFIX) and not header.endswith(SUFFIX_TO_SKIP)
    ]

print('Boolean columns:')
print('- ' + '\n- '.join(bool_headers))

detailed_data = {}
for header in bool_headers:
  selected_images = imagelab.issues[imagelab.issues[header]]
  selected_image_files = selected_images.index.tolist()
  detailed_data[header] = convert_list_to_image_indices(selected_image_files)

#### Export to JSON

In [ ]:
fname = 'detailed_issues.json'
save_to_json(detailed_data, fname)

In [ ]:
imagelab.issues[header]

### Duplicate sets

> `imagelab.info` can be used to retrieve **which images** are near or exact duplicates of each other.
>
> References:
> - https://cleanvision.readthedocs.io/en/latest/tutorials/tutorial.html#imagelab.info
> - https://cleanvision.readthedocs.io/en/latest/tutorials/tutorial.html#Duplicate-sets

#### Count duplicate sets

In [ ]:
exact_duplicates = convert_nested_lists_to_image_indices(imagelab.info['exact_duplicates']['sets'])
print(f"[exact duplicates] #sets = {len(exact_duplicates)}")

near_duplicates = convert_nested_lists_to_image_indices(imagelab.info['near_duplicates']['sets'])
print(f"[near duplicates] #sets = {len(near_duplicates)}")

#### Export to JSON

In [ ]:
for field in ['exact_duplicates', 'near_duplicates']:
  fname = f'{field}.json'
  data = convert_nested_lists_to_image_indices(imagelab.info[field]['sets'])
  save_to_json(data, fname)